In [1]:
import datetime

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [2]:
import warnings
warnings.filterwarnings('ignore')

plt.rcParams.update({'font.size': 22})
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.options.mode.chained_assignment = None

In [3]:
scraped_df = pd.read_csv('raw_scraped_data.csv')
scraped_df_f = scraped_df[scraped_df['raw_history'].notna()]
len(scraped_df_f) / len(scraped_df)

0.9993744267177359

In [4]:
len(scraped_df_f)

260398

In [5]:
scraped_df.head()

,external_id,beds,baths,sqft,property_type,raw_history
0,13959615,7,3,-,Multi-Family (2-4 Unit),"[{'event': 'Sold (MLS)', 'date': datetime.date..."
1,13398829,9,3,-,Multi-Family (2-4 Unit),"[{'event': 'Price Changed', 'date': datetime.d..."
2,13236623,6,3,-,Multi-Family (2-4 Unit),"[{'event': 'Contingent', 'date': datetime.date..."
3,17619341,5,2.5,"2,880",Single Family Residential,"[{'event': 'Sold (Public Records)', 'date': da..."
4,13809758,4,2.5,"1,638",Single Family Residential,"[{'event': 'Sold (Public Records)', 'date': da..."


In [6]:
scraped_df_f['raw_history'] = scraped_df_f['raw_history'].apply(lambda x: eval(x))

In [7]:
listing_info_df = pd.read_csv('final_redfin_listings_info.csv')
listing_info_df['created_date'] = listing_info_df['created_date'].apply(
        lambda date: datetime.datetime.strptime(date, '%Y-%m-%d').date())

In [8]:
combined_df = listing_info_df.merge(scraped_df_f, on='external_id', how='inner')

In [9]:
len(combined_df)

256742

In [10]:
len(listing_info_df)

264139

In [11]:
len(combined_df) / len(listing_info_df)

0.9719958052389083

In [12]:
combined_df.head()

,external_id,created_date,address,city,state,zipcode,market_area,null_redfin_price_estimate,buyside_commission_range,created_month_index,price_range,year_built_range,price_difference_pct_range,avg_views_5,avg_views_10,avg_views_30,avg_views_100,avg_views_200,redfin_price_estimate,opendoor,zillow,offerpad,hoa,beds_range,baths_range,sqft_range,property_category_index,brokerage_index,brokerage_listings_pct_range,beds,baths,sqft,property_type,raw_history
0,13663942,2022-01-14,10 Terrace Ln,Des Plaines,IL,60019,Chicago,1,0,7,6,4,1,1888.153755,1684.743534,1518.060252,1231.980749,1049.688773,NaN,0,0,0,0,3,2,1,0,0,8,4,2,"1,273",Single Family Residential,"[{'event': 'Sold (Public Records)', 'date': 20..."
1,23331416,2021-07-27,208 Meadowbrook Dr,Bolingbrook,IL,60440,Chicago,1,0,1,3,4,1,1243.320956,1097.936268,1113.508900,1262.284556,1204.498678,NaN,0,0,0,0,2,2,3,0,0,8,3,2,"1,811",Single Family Residential,"[{'event': 'Sold (Public Records)', 'date': 20..."
2,13847488,2021-08-03,303 E Ivy Ln,Arlington Heights,IL,60004,Chicago,1,0,2,6,4,1,860.582481,943.937557,938.306499,1243.650506,1115.046550,NaN,0,0,0,0,2,2,1,0,0,8,3,2,"1,540",Single Family Residential,"[{'event': 'Sold (Public Records)', 'date': 20..."
3,21765334,2021-11-10,202 Hemlock Ave,Romeoville,IL,60446,Chicago,1,0,5,2,4,1,896.332022,1171.125672,986.257433,896.232247,992.979730,NaN,0,0,0,0,3,2,0,0,0,8,4,2,"1,170",Single Family Residential,"[{'event': 'Sold (Public Records)', 'date': 20..."
4,13894291,2021-07-20,458 E Carpenter Dr,Palatine,IL,60074,Chicago,1,0,1,7,4,1,923.631107,1012.231276,915.941123,927.873459,803.631951,NaN,0,0,0,0,3,3,4,0,0,8,4,3.5,"2,468",Single Family Residential,"[{'event': 'Sold (MLS)', 'date': 2021-08-26, '..."


In [13]:
def get_event_type(text):
    if any(phrase in text for phrase in {'Pending', 'Under Contract', 'Contingent'}):
        return 'pending'
    if 'Sold' in text:
        return 'sold'
    if any(phrase in text for phrase in {'Listed', 'Relisted'}):
        return 'listed'
    if 'Listing Removed' in text:
        return 'listing_removed'
    if 'Coming Soon' in text:
        return 'coming_soon'
    return None

In [14]:
def get_listing_history(row):
    redfin_events = row['raw_history']
    scraped_date = row['created_date']
    
    event_type_to_date = {}
    event_lst = []
    subtract_from_dom = 0
    sale_price = None
    for i, redfin_event in enumerate(redfin_events):
        event_type = get_event_type(redfin_event['event'])
        event_date = redfin_event['date']

        # If the listing was scarped after the sale date, it must be that this sale date 
        # is referring to a previous transaction of the given property
        if event_type == 'sold' and scraped_date > event_date:
            return ['no_sale'] * 6
        
        # similarly, if the listing was scraped more than 9 days after the listing went under contract,
        # this pending date is almost certaintly referring to a previous transaction of the given property 
        if event_type == 'pending' and scraped_date > event_date + datetime.timedelta(days=9):
            return ['no_sale'] * 6

        if event_type == 'sold':
            event_type_to_date = {}
            event_lst = []
            sale_price = redfin_event.get('price')

        if event_type in {'sold', 'pending'}:
            event_type_to_date[event_type] = event_date

        coming_soon_scenario = False
        
        # check if listing was scraped while the listing was in the coming soon phase
        try:
            for j in [1, 2]:
                next_event = redfin_events[i + j]
                next_event_type = get_event_type(next_event['event'])
                if next_event_type == 'coming_soon':
                    coming_soon_date = next_event['date']
                    if scraped_date >= coming_soon_date:
                        coming_soon_scenario = True
                        break
                        
        except IndexError:
            pass
        
        # check that the listing was scraped while the listing was in the coming soon phase 
        if event_type == 'listed' and (scraped_date >= event_date or coming_soon_scenario):
            # check that the given listing went under contract and ended up selling within 120 days of going
            # under contract
            if (all(event_type in event_type_to_date for event_type in {'sold', 'pending'}) and
                (event_type_to_date['sold'] - event_type_to_date['pending']).days < 120):
                list_date = event_date
                list_to_pending_days = (event_type_to_date['pending'] - list_date).days + 1

                removed_df = pd.DataFrame(event_lst, columns=['event', 'date'])
                
                # check if property was delisted and, if so, calculate the number of days it was delisted
                unique_events = set(removed_df['event'])
                if all(event in unique_events for event in {'listed', 'listing_removed'}):
                    removed_df = removed_df.sort_values('date').reset_index(drop=True)
                    order_of_events = ['listing_removed', 'listed']
                    
                    # only consider times when property was delisted between list date and pending date
                    removed_df = removed_df[(list_date < removed_df['date']) & 
                                            (removed_df['date'] < event_type_to_date['pending'])]
                    
                    # iterate through each time the property was delisted
                    nrow = len(removed_df)
                    if nrow % 2 == 0:
                        for i in range(int(nrow / 2)):
                            df_f = removed_df.iloc[2 * i: 2 * i + 2]
                            events = list(df_f['event'])
                            if events == order_of_events:
                                dates = list(df_f['date'])
                                subtract_from_dom += (dates[1] - dates[0]).days

                dom = list_to_pending_days - subtract_from_dom
                
                return [list_date, event_type_to_date['pending'], event_type_to_date['sold'], 
                        list_to_pending_days, dom, sale_price]
            
        elif event_type in {'listed', 'listing_removed'}:
            event_lst.append([event_type, event_date])
    
    return ['no_sale'] * 6 

In [15]:
new_cols = ['list_date', 'pending_date', 'sale_date', 'list_to_pending_days', 'dom', 'sale_price']

combined_df[new_cols] = combined_df.apply(get_listing_history, axis=1, result_type='expand')

In [16]:
def get_days_until_sale(row):
    
    scraped_date = row['created_date']
    redfin_events = row['raw_history']
    
    was_listed = False
    
    for redfin_event in redfin_events:
        event_type = get_event_type(redfin_event['event'])
        event_date = redfin_event['date']
        
        # check that the listing was scraped within 30 days of the property being listed
        if event_type == 'listed' and abs((event_date - scraped_date).days) < 30:
            was_listed = True
            break
    
    # if the property was never listed, we exclude it from the analysis
    if not was_listed:
        return None
    
    sold_date = None
    
    for redfin_event in redfin_events:
        event_type = get_event_type(redfin_event['event'])
        event_date = redfin_event['date']

        if event_type == 'sold':
            sold_date = event_date
            
            # If the listing was scarped after the sale date, it must be that this sale date 
            # is referring to a previous transaction of the given property
            if scraped_date > sold_date:
                return 'no_sale'
            
        if event_type == 'listed' and scraped_date >= event_date:
            if sold_date is None:
                return 'no_sale'
            return (sold_date - event_date).days
    
    return 'no_sale'

In [17]:
combined_df['days_until_sale'] = combined_df.apply(get_days_until_sale, axis=1)

In [18]:
days_to_sell_threshold = 120

def did_home_sell(days_until_sale):
    if pd.isnull(days_until_sale):
        return None
    if days_until_sale == 'no_sale':
        return 0
    return int(days_until_sale < days_to_sell_threshold)

In [19]:
combined_df['home_sold'] = combined_df['days_until_sale'].apply(did_home_sell)

In [20]:
combined_df['home_sold'].value_counts(dropna=False) / len(combined_df)

1.0    0.701338
0.0    0.229308
NaN    0.069354
Name: home_sold, dtype: float64

In [21]:
final_df = combined_df.drop(columns=['raw_history'])
final_df.shape

(256742, 41)

In [22]:
final_df.to_csv('sale_outcome_info.csv', index=False)